**5.**

In [50]:
!pip install snowflake-connector-python pandas scikit-learn


**CONNECTION WITH SNOWFLAKE**

In [51]:
import snowflake.connector
import pandas as pd


conn = snowflake.connector.connect(
    user='',
    password='',
    account='',
    warehouse='',
    database='',
    schema=''
)


cursor = conn.cursor()


query = "SELECT * FROM TITANIC LIMIT 1000"
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])




In [4]:
df.head()

,PASSENGERID,SURVIVED,PCLASS,NAME,SEX,AGE,SIBSP,PARCH,TICKET,FARE,CABIN,EMBARKED
0,1,0,3,"Braund, Mr. Owen Harris",male,22.00,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.00,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.00,0,0,373450,8.0500,None,S


In [52]:
df.columns

Index(['PASSENGERID', 'SURVIVED', 'PCLASS', 'NAME', 'SEX', 'AGE', 'SIBSP',
       'PARCH', 'TICKET', 'FARE', 'CABIN', 'EMBARKED'],
      dtype='object')

**FEATURE ENGINEERING**

In [53]:
print(df.isnull().sum())

PASSENGERID      0
SURVIVED         0
PCLASS           0
NAME             0
SEX              0
AGE            177
SIBSP            0
PARCH            0
TICKET           0
FARE             0
CABIN          687
EMBARKED         2
dtype: int64


In [54]:
for col in df.columns:
    print(f"\nColumn: {col}")
    print(df[col].value_counts(dropna=False))


Column: PASSENGERID
PASSENGERID
891    1
1      1
2      1
3      1
4      1
      ..
16     1
15     1
14     1
13     1
12     1
Name: count, Length: 891, dtype: int64

Column: SURVIVED
SURVIVED
0    549
1    342
Name: count, dtype: int64

Column: PCLASS
PCLASS
3    491
1    216
2    184
Name: count, dtype: int64

Column: NAME
NAME
Dooley, Mr. Patrick                                    1
Braund, Mr. Owen Harris                                1
Cumings, Mrs. John Bradley (Florence Briggs Thayer)    1
Heikkinen, Miss. Laina                                 1
Futrelle, Mrs. Jacques Heath (Lily May Peel)           1
                                                      ..
Hewlett, Mrs. (Mary D Kingcome)                        1
Vestrom, Miss. Hulda Amanda Adolfina                   1
Andersson, Mr. Anders Johan                            1
Saundercock, Mr. William Henry                         1
Bonnell, Miss. Elizabeth                               1
Name: count, Length: 891, dtype: int

In [55]:
df = df[df['EMBARKED'].notna()]

In [56]:
df['AGE'] = df['AGE'].fillna(df['AGE'].median())

In [57]:
from sklearn.preprocessing import LabelEncoder
df['SEX'] = LabelEncoder().fit_transform(df['SEX'])
df = pd.get_dummies(df, columns=['EMBARKED'], prefix='EMBARKED')
df['FAMILYSIZE'] = df['SIBSP'] + df['PARCH'] + 1
df['ISALONE'] = (df['FAMILYSIZE'] == 1).astype(int)
df['TITLE'] = df['NAME'].str.extract(' ([A-Za-z]+)\.', expand=False)
df['TITLE'] = df['TITLE'].replace(
    ['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'],
    'Rare'
)
df['TITLE'] = df['TITLE'].replace({'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs'})
df['TITLE'] = LabelEncoder().fit_transform(df['TITLE'])
df = df.drop(columns=['PASSENGERID', 'NAME', 'TICKET', 'CABIN'])

In [58]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[['AGE', 'FARE', 'FAMILYSIZE']] = scaler.fit_transform(df[['AGE', 'FARE', 'FAMILYSIZE']])


In [66]:
df.head()

,SURVIVED,PCLASS,SEX,AGE,FARE,EMBARKED_C,EMBARKED_Q,EMBARKED_S,FAMILYSIZE,ISALONE,TITLE
0,0,3,1,-0.563674,-0.500240,False,False,True,0.057853,0,2
1,1,1,0,0.669217,0.788947,True,False,False,0.057853,0,3
2,1,3,0,-0.255451,-0.486650,False,False,True,-0.561804,1,1
3,1,1,0,0.438050,0.422861,False,False,True,0.057853,0,3
4,0,3,1,0.438050,-0.484133,False,False,True,-0.561804,1,2


In [68]:
print(df.columns.tolist())
features_list=df.columns.tolist()



['SURVIVED', 'PCLASS', 'SEX', 'AGE', 'FARE', 'EMBARKED_C', 'EMBARKED_Q', 'EMBARKED_S', 'FAMILYSIZE', 'ISALONE', 'TITLE']


In [64]:

features_list=df.columns.tolist()

**STORING IT IN SNOWFLAKE FS**

In [70]:
from snowflake.connector.pandas_tools import write_pandas

success, nchunks, nrows, _ = write_pandas(
    conn=conn,
    df=df_to_upload,
    table_name='TITANIC_FEATURES_FINAL',  # use an existing or new table name
    auto_create_table=True
)


<ipython-input-70-b3181b7b4f71>:3: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, nchunks, nrows, _ = write_pandas(


In [71]:
print(success)
print(nrows)

True
889


**IMPORTING FEATURES**

In [73]:
query = "SELECT * FROM TITANIC_FEATURES_FINAL"
df = pd.read_sql(query, conn)

<ipython-input-73-81b93052b21b>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [74]:
df.head()

,SURVIVED,PCLASS,SEX,AGE,FARE,EMBARKED_C,EMBARKED_Q,EMBARKED_S,FAMILYSIZE,ISALONE,TITLE
0,0,3,1,-0.563674,-0.500240,False,False,True,0.057853,0,2
1,1,1,0,0.669217,0.788947,True,False,False,0.057853,0,3
2,1,3,0,-0.255451,-0.486650,False,False,True,-0.561804,1,1
3,1,1,0,0.438050,0.422861,False,False,True,0.057853,0,3
4,0,3,1,0.438050,-0.484133,False,False,True,-0.561804,1,2


In [76]:
df.head()

,SURVIVED,PCLASS,SEX,AGE,FARE,EMBARKED_C,EMBARKED_Q,EMBARKED_S,FAMILYSIZE,ISALONE,TITLE
0,0,3,1,-0.563674,-0.500240,False,False,True,0.057853,0,2
1,1,1,0,0.669217,0.788947,True,False,False,0.057853,0,3
2,1,3,0,-0.255451,-0.486650,False,False,True,-0.561804,1,1
3,1,1,0,0.438050,0.422861,False,False,True,0.057853,0,3
4,0,3,1,0.438050,-0.484133,False,False,True,-0.561804,1,2


In [77]:
print(df.isnull().sum())

SURVIVED      0
PCLASS        0
SEX           0
AGE           0
FARE          0
EMBARKED_C    0
EMBARKED_Q    0
EMBARKED_S    0
FAMILYSIZE    0
ISALONE       0
TITLE         0
dtype: int64


In [78]:

X = df.drop(columns=['SURVIVED'])
y = df['SURVIVED']


**PERFROMING ML TASK**

In [79]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print(f"Model accuracy: {accuracy * 100:.2f}%")


Model accuracy: 80.34%


In [83]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [84]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7471910112359551
Confusion Matrix:
 [[80 29]
 [16 53]]
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.73      0.78       109
           1       0.65      0.77      0.70        69

    accuracy                           0.75       178
   macro avg       0.74      0.75      0.74       178
weighted avg       0.76      0.75      0.75       178



In [86]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

param_grid = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'criterion': ['gini', 'entropy']
}

In [91]:
dt = DecisionTreeClassifier(random_state=42)


grid_search = GridSearchCV(dt, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)


best_dt = grid_search.best_estimator_


from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = best_dt.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.7865168539325843
Confusion Matrix:
 [[89 20]
 [18 51]]
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.82      0.82       109
           1       0.72      0.74      0.73        69

    accuracy                           0.79       178
   macro avg       0.78      0.78      0.78       178
weighted avg       0.79      0.79      0.79       178

